In [63]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from fuzzywuzzy import fuzz
from datetime import date
import datetime as dt
import pandas as pd
import numpy as np
import random
from selenium.webdriver.chrome.options import Options
import re
from tqdm import tqdm
import warnings
import selenium
warnings.filterwarnings("ignore")

In [64]:
driver = webdriver.Chrome()
driver.get("https://www.wnba.com/schedule?season=2024&month=all")
time.sleep(5)

In [73]:
row_text = [val.text for val in driver.find_elements("tag name","a")]

In [74]:
date_series = pd.Series([val.text for val in driver.find_elements("tag name","h2")])
date_series = date_series[date_series.str.contains("2024")].apply(lambda x: x.split(", "))

date_series = date_series.apply(lambda x: x[-2]+", "+x[-1])
date_series = pd.to_datetime(date_series.apply(lambda x: dt.date.strftime(dt.datetime.strptime(x, "%b %d, %Y"),"%Y-%m-%d")))

In [75]:
span_tag = pd.Series([val.text for val in driver.find_elements("tag name","span")])
game_numbers = span_tag[span_tag.str.contains("Game")]


In [76]:
game_numbers = list(game_numbers.apply(lambda x: int(x.split(" ")[0])))

In [77]:
all_date = []

for date, number in zip(date_series,game_numbers):
    
    for i in range(number):
        all_date.append(date)
    
    

In [78]:
#taking out the preseason game
all_date = all_date[1:]

In [79]:
row_series = pd.Series(row_text)[pd.Series(row_text).str.contains("GMT")]

In [83]:
location = list(row_series.apply(lambda x: x.split("\n")[-1]))
away_team = list(row_series.apply(lambda x: x.split("\n")[1]))
home_team = list(row_series.apply(lambda x: x.split("\n")[2]))
gametime = list(row_series.apply(lambda x: x.split("\n")[0]))


In [85]:
full_df = pd.DataFrame(zip(all_date,home_team,away_team,location,gametime),columns=["GAME_DATE","HOME_TEAM","AWAY_TEAM","LOCATION","GAME_TIME"])

In [86]:
full_df.to_csv("wnba_schedule.csv",index=False)

In [87]:
full_df

,GAME_DATE,HOME_TEAM,AWAY_TEAM,LOCATION,GAME_TIME
0,2024-05-14,Washington Mystics,New York Liberty,"Entertainment and Sports Arena Washington, DC",8:00PM GMT-3
1,2024-05-14,Connecticut Sun,Indiana Fever,"Mohegan Sun Arena Uncasville, CT",9:00PM GMT-3
2,2024-05-14,Las Vegas Aces,Phoenix Mercury,"Michelob ULTRA Arena Las Vegas, NV",11:00PM GMT-3
3,2024-05-14,Seattle Storm,Minnesota Lynx,"Climate Pledge Arena Seattle, WA",11:00PM GMT-3
4,2024-05-15,Dallas Wings,Chicago Sky,"College Park Center Arlington, TX",9:00PM GMT-3
...,...,...,...,...,...
236,2024-09-19,New York Liberty,Atlanta Dream,"Barclays Center Brooklyn, NY",8:00PM GMT-3
237,2024-09-19,Washington Mystics,Indiana Fever,"Entertainment and Sports Arena Washington, DC",8:00PM GMT-3
238,2024-09-19,Minnesota Lynx,Los Angeles Sparks,"Target Center Minneapolis, MN",8:00PM GMT-3
239,2024-09-19,Las Vegas Aces,Dallas Wings,"Michelob ULTRA Arena Las Vegas, NV",10:00PM GMT-3
